In [ ]:
import pandas as pd
import numpy as np
import datetime
import calendar
from calendar import monthrange
from datetime import datetime, timedelta


def blank_ts_df(dataframe, year_column_index, month_column_index):

    df = dataframe

    total_time_df = pd.DataFrame()

    start_yr = int(df.iloc[:, year_column_index].min())
    end_yr = int(df.iloc[:, year_column_index].max())


    year_list = [y for y in range(start_yr, end_yr+1 , 1)]

    for yr in year_list:
        
        if yr == end_yr:
            end_month = int(df.iloc[:, month_column_index].max())
            months_list = [m for m in range(1, end_month+1, 1)]

        else:
            months_list = [m for m in range(1, 12+1 , 1)]
        
        yearly_time_df = pd.DataFrame()


        for month in months_list:

            start_end_time_list = []

            end_day = monthrange(yr, month)[1]

            start_date = (datetime(year= yr, month= month, day=1, hour=0, minute=0, second =0)).strftime('%Y-%m-%d %H:%M:%S')
            end_date = (datetime(year= yr, month= month, day= end_day, hour=21, minute=0, second =0)).strftime('%Y-%m-%d %H:%M:%S')

            monthly_time_list = pd.date_range(start_date, end_date, freq="3H")

            monthly_time_df = pd.DataFrame({'Date_Time':monthly_time_list})

            yearly_time_df = pd.concat([yearly_time_df,monthly_time_df])

        total_time_df = pd.concat([total_time_df,yearly_time_df])

    return total_time_df


full_df = pd.read_csv(r"E:\Script\BMD_Wind_Data_Processing\New folder\Three Hourly Wind.csv")

station_name_index_df = full_df[full_df['Unnamed: 0'].str.contains('Station', na=False)]

station_name_list = list(station_name_index_df['Unnamed: 0'].map(lambda x: (x.split(':')[1]).split(' ')[1]))
station_start_index_dict = {station:index for station, index in zip(station_name_list, station_name_index_df.index)}

identifier_index_list = list(full_df.iloc[11, :4])

for i, station_name in zip(np.arange(1, len(station_name_list)+1), station_name_list):
    
    if i == len(station_name_list):
        
        start_index = station_start_index_dict[station_name_list[i-1]] + 7
        end_index = len(full_df)
        
        # print('i = ' + str(i) + '  '+str(start_index) +  '  '+str(end_index))
        
    else:
        start_index = station_start_index_dict[station_name_list[i-1]] + 7
        end_index = station_start_index_dict[station_name_list[i]] - 3
    
    
    station_raw_df = full_df.iloc[start_index : end_index+1, :]
    
    wind_speed_col_list = [name for name in (station_raw_df.columns[4:]) if int(name.split(':')[1]) % 2 == 0]
    wind_direction_col_list = [name for name in (station_raw_df.columns[4:]) if int(name.split(':')[1]) % 2 != 0]

    df_sp = station_raw_df[list(station_raw_df.columns[:4]) + wind_speed_col_list]
    df_dr = station_raw_df[list(station_raw_df.columns[:4]) + wind_direction_col_list]
   
    df_sp.columns = identifier_index_list + list(np.arange(1, len(df_sp.columns[4:]) + 1))
    df_dr.columns = identifier_index_list + list(np.arange(1, len(df_dr.columns[4:]) + 1))


    wind_speed_df = df_sp.melt(id_vars=list(df_sp.columns[0:4]), value_vars= list(df_sp.columns[4:]),
                             var_name='Day', value_name='Wind Speed (knots)')

    wind_direction_df = df_dr.melt(id_vars=list(df_dr.columns[0:4]), value_vars= list(df_dr.columns[4:]),
                                   var_name='Day', value_name='Wind Direction (degree)')


    wind_speed_df['Date_Time'] = pd.to_datetime(dict(year=wind_speed_df['Year'], month=wind_speed_df['Month'], 
                                                     day=wind_speed_df['Day'],hour=wind_speed_df['Tm']), 
                                                format= '%Y-%m-%d %H:%M:%S', errors='coerce')

    wind_direction_df['Date_Time'] = pd.to_datetime(dict(year=wind_direction_df['Year'], month=wind_direction_df['Month'], 
                                                         day=wind_direction_df['Day'],hour=wind_direction_df['Tm']), 
                                                    format= '%Y-%m-%d %H:%M:%S', errors='coerce')

    wind_speed_df.dropna(axis =0, subset = ['Date_Time'], inplace = True)
    wind_direction_df.dropna(axis =0, subset = ['Date_Time'], inplace = True)


    # speed_direction_df = wind_speed_df.merge(wind_direction_df, how="left")

    speed_direction_df = wind_speed_df.merge(wind_direction_df, how="left", on="Date_Time",
                                             suffixes=('', '_y'))

    speed_direction_df.drop(speed_direction_df.filter(regex='_y$').columns, axis=1, inplace=True)

    total_time_df = blank_ts_df(station_raw_df, year_column_index =1 , month_column_index =2 )

    station_ts_df = total_time_df.merge(speed_direction_df, how="left", on="Date_Time")
    station_ts_df = station_ts_df[['Index', 'Year', 'Month','Day','Tm','Date_Time','Wind Speed (knots)', 'Wind Direction (degree)']]
    station_ts_df.insert(loc = 1, column = 'Station_Name', value = station_name)
    station_ts_df.to_csv('Output/{}_3H_Wind.csv'.format(station_name), index = False)